In [ ]:
#pip install lightning

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
PATH = '/content/drive/MyDrive/ML Cup 2023 Weather/train/'

In [ ]:
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import lightning as L
import tqdm

In [ ]:
import random
from lightning import seed_everything
seed_everything(7, workers=True)
seed=7
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

INFO: Seed set to 7
INFO:lightning.fabric.utilities.seed:Seed set to 7


In [ ]:
class ConvLSTMCell(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, padding, activation):
        super().__init__()

        if activation == 'tanh':
            self.activation = torch.tanh
        elif activation == 'relu':
            self.activation = torch.relu

        self.conv_1 = nn.Conv2d(
            in_channels=in_channels + out_channels,
            out_channels= 4 * out_channels,
            kernel_size=kernel_size,
            padding= padding
        )
        self.conv_2 = nn.Conv2d(
            in_channels=in_channels + out_channels,
            out_channels= 4 * out_channels,
            kernel_size=kernel_size,
            padding= padding
        )
        self.conv_3 = nn.Conv2d(
            in_channels=in_channels + out_channels,
            out_channels= 4 * out_channels,
            kernel_size=kernel_size,
            padding= padding
        )
        self.conv_wind = nn.Conv2d(
            in_channels= 10 + out_channels,
            out_channels= 4 * out_channels,
            kernel_size=kernel_size,
            padding= padding
        )

        self.conv_int = nn.Conv2d(
            in_channels=in_channels + out_channels,
            out_channels=4 * out_channels,
            kernel_size=kernel_size,
            padding=padding
        )

    def forward(self, X, H_prev, C_prev):
        int_X = X[:,0:1, :, :]
        wind_X= X[:,1:, :, :]
        int_H_prev = H_prev[:,0:32, :, :]
        wind_H_prev = H_prev[:,32:, :, :]

        conv_int_output = self.conv_int(torch.cat([int_X, int_H_prev], dim=1))
        conv_wind_output = self.conv_wind(torch.cat([wind_X, wind_H_prev], dim=1))

        i_conv_int, f_conv_int, C_conv_int, o_conv_int = torch.chunk(conv_int_output, chunks=4, dim=1)
        i_conv_wind, f_conv_wind, C_conv_wind, o_conv_wind = torch.chunk(conv_wind_output, chunks=4, dim=1)

        input_gate = torch.sigmoid(torch.cat([i_conv_int, i_conv_wind], dim=1))
        forget_gate = torch.sigmoid(torch.cat([f_conv_int, f_conv_wind], dim=1))
        output_gate = torch.sigmoid(torch.cat([o_conv_int,  o_conv_wind], dim=1))
        C_conv = torch.cat([C_conv_int, C_conv_wind], dim=1)
        C = forget_gate * C_prev + input_gate * self.activation(C_conv)
        H = output_gate * self.activation(C)


        '''X.shape: torch.Size([1, 4, 252, 252])
H_prev.shape: torch.Size([1, 128, 252, 252])
C_prev.shape: torch.Size([1, 128, 252, 252])
int_X.shape: torch.Size([1, 1, 252, 252])
conv_int_output.shape: torch.Size([1, 128, 252, 252])
i_conv_int.shape: torch.Size([1, 32, 252, 252])
input_gate.shape: torch.Size([1, 128, 252, 252])
H.shape: torch.Size([1, 128, 252, 252])
C.shape: torch.Size([1, 128, 252, 252])'''

        return H, C


class ConvLSTM(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, padding, activation):
        super().__init__()
        self.out_channels = out_channels
        self.convLSTMCell = ConvLSTMCell(in_channels, out_channels, kernel_size, padding, activation)

    def forward(self, X):
        batch_size, seq_len, _, height, width = X.size()
        output = torch.zeros(batch_size, seq_len, 2 * self.out_channels, height, width, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        H = torch.zeros(batch_size, 2 * self.out_channels, height, width, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        C = torch.zeros(batch_size, 2 * self.out_channels, height, width, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        for time_step in range(seq_len):
            H, C = self.convLSTMCell(X[:, time_step], H, C)
            output[:, time_step] = H
        return output


class Seq2Seq(nn.Module):

    def __init__(
        self, num_channels, num_kernels, kernel_size, padding, activation, num_layers, out_seq_len
    ):
        super().__init__()
        self.out_seq_len = out_seq_len

        self.activation = torch.relu
        self.sequential = nn.Sequential()
        self.sequential.add_module(
            'convlstm1',
            ConvLSTM(
                in_channels=num_channels,
                out_channels=num_kernels,
                kernel_size=kernel_size,
                padding=padding,
                activation=activation
            )
        )
        for layer_index in range(2, num_layers + 1):
            self.sequential.add_module(
                f'convlstm{layer_index}',
                ConvLSTM(
                    in_channels=num_kernels,
                    out_channels=num_kernels,
                    kernel_size=kernel_size,
                    padding=padding,
                    activation=activation
                )
            )
        self.conv = nn.Conv2d(
            in_channels=2*num_kernels,
            out_channels=num_channels,
            kernel_size=kernel_size,
            padding=padding
        )

    def forward(self, X):
        batch_size, seq_len, num_channels, height, width = X.size()
        inputs = torch.zeros(
            batch_size, seq_len + self.out_seq_len - 1, num_channels, height, width,
            device=self.conv.weight.device
        )
        inputs[:, :seq_len] = X
        output = self.sequential(inputs)
        output = torch.stack([
            self.conv(output[:, index + seq_len - 1])
            for index in range(self.out_seq_len)
        ], dim=1)
        return output


class ConvLSTMModel(L.LightningModule):

    def __init__(self):
        super().__init__()
        self.model = Seq2Seq(
            num_channels=1,
            num_kernels=32,
            kernel_size=(3, 3),
            padding=(1, 1),
            activation='relu',
            num_layers=1,
            out_seq_len=12
        )

    def forward(self, x):
        x = x.to(device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        output = self.model(x)
        return output

    def training_step(self, batch):
        x, y = batch
        out = self.forward(x)
        out[y == -1] = -1
        loss = F.mse_loss(out, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-4)
        return optimizer

In [ ]:
class RadarDataset_0(data.Dataset):

    def __init__(self, list_of_files, in_seq_len=4, out_seq_len=12, mode='sequentially', with_time=False):
        self.in_seq_len = in_seq_len
        self.out_seq_len = out_seq_len
        self.seq_len = in_seq_len + out_seq_len
        self.with_time = with_time
        self.__prepare_timestamps_mapping(list_of_files)
        self.__prepare_sequences(mode)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        to_append = []
        data = []
        targets = []
        for timestamp in self.sequences[index]:
            with h5py.File(self.timestamp_to_file[timestamp]) as d:
                targets.append(np.array(d[timestamp]['intensity']))
                data.append(np.array([d[timestamp]['intensity'], d[timestamp]['radial_velocity'][0],
                                      d[timestamp]['radial_velocity'][1], d[timestamp]['radial_velocity'][2],
                                      d[timestamp]['radial_velocity'][3], d[timestamp]['radial_velocity'][4],
                                      d[timestamp]['radial_velocity'][5], d[timestamp]['radial_velocity'][6],
                                      d[timestamp]['radial_velocity'][7], d[timestamp]['radial_velocity'][8],
                                      d[timestamp]['radial_velocity'][9]]))
                '''
                for i in range(1):
                    to_append.append(np.array(d[timestamp]['radial_velocity'][i]))'''
        data = np.array(data)
        #data = np.expand_dims(data, axis=1)
        targets = np.expand_dims(targets, axis=1)
        data[data == -1e6] = 0
        data[data == -2e6] = -1
        targets[targets == -1e6] = 0
        targets[targets == -2e6] = -1
        inputs = data[:self.in_seq_len]
        targets = targets[self.in_seq_len:]
        if self.with_time:
            return (inputs, self.sequences[index][-1]), targets
        else:
            return inputs, targets

    def __prepare_timestamps_mapping(self, list_of_files):
        self.timestamp_to_file = {}
        for filename in list_of_files:
            with h5py.File(filename) as d:
                self.timestamp_to_file = {
                    **self.timestamp_to_file,
                    **dict(map(lambda x: (x, filename), d.keys()))
                }

    def __prepare_sequences(self, mode):
        timestamps = np.unique(sorted(self.timestamp_to_file.keys()))
        if mode == 'sequentially':
            self.sequences = [
                timestamps[index * self.seq_len: (index + 1) * self.seq_len]
                for index in range(len(timestamps) // self.seq_len)
            ]
        elif mode == 'overlap':
            self.sequences = [
                timestamps[index: index + self.seq_len]
                for index in range(len(timestamps) - self.seq_len + 1)
            ]
        else:
            raise Exception(f'Unknown mode {mode}')
        self.sequences = list(filter(
            lambda x: int(x[-1]) - int(x[0]) == (self.seq_len - 1) * 600,
            self.sequences
        ))

In [ ]:
def prepare_loaders(train_batch_size=1):
    train_dataset_0 = RadarDataset_0([PATH + '2021-03-train.hdf5'])

    '''
    train_dataset_0 = RadarDataset_0([
        PATH + '2021-01-train.hdf5', PATH + '2021-03-train.hdf5', PATH + '2021-04-train.hdf5',
        PATH + '2021-06-train.hdf5', PATH + '2021-07-train.hdf5', PATH + '2021-09-train.hdf5',
        PATH + '2021-10-train.hdf5', PATH + '2021-12-train.hdf5'])
    train_dataset_1 = RadarDataset_1([
        PATH + '2021-01-train.hdf5', PATH + '2021-03-train.hdf5', PATH + '2021-04-train.hdf5',
        PATH + '2021-06-train.hdf5', PATH + '2021-07-train.hdf5', PATH + '2021-09-train.hdf5',
        PATH + '2021-10-train.hdf5', PATH + '2021-12-train.hdf5'])
    train_dataset_2 = RadarDataset_2([
        PATH + '2021-01-train.hdf5', PATH + '2021-03-train.hdf5', PATH + '2021-04-train.hdf5',
        PATH + '2021-06-train.hdf5', PATH + '2021-07-train.hdf5', PATH + '2021-09-train.hdf5',
        PATH + '2021-10-train.hdf5', PATH + '2021-12-train.hdf5'])
    train_dataset_3 = RadarDataset_3([
        PATH + '2021-01-train.hdf5', PATH + '2021-03-train.hdf5', PATH + '2021-04-train.hdf5',
        PATH + '2021-06-train.hdf5', PATH + '2021-07-train.hdf5', PATH + '2021-09-train.hdf5',
        PATH + '2021-10-train.hdf5', PATH + '2021-12-train.hdf5'])
    train_dataset = torch.utils.data.ConcatDataset([train_dataset_0, train_dataset_1, train_dataset_2, train_dataset_3])'''
    train_datasets = torch.utils.data.random_split(train_dataset_0, [0.2, 0.2, 0.2, 0.2, 0.2])
    train_loaders = []
    for i in range(5):
        train_loaders.append(data.DataLoader(train_datasets[i], batch_size=train_batch_size, shuffle=True, num_workers=2))
    return train_loaders

def prepare_valid_loader(valid_batch_size=1):
    valid_dataset = RadarDataset_0([PATH + '2021-08-train.hdf5'])
    #valid_dataset = RadarDataset_0([PATH + '2021-08-train.hdf5', PATH + '2021-05-train.hdf5', PATH + '2021-02-train.hdf5', PATH + '2021-11-train.hdf5'])
    valid_loader = data.DataLoader(valid_dataset, batch_size=valid_batch_size, shuffle=False, num_workers=2)
    return valid_loader

def prepare_test_loader(test_batch_size=1):
    test_dataset = RadarDataset_0([PATH + '2022-test-public.hdf5'], out_seq_len=0, with_time=True)
    test_loader = data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)
    return test_loader

def evaluate_on_val(model, valid_loader):
    rmses = np.zeros((12,), dtype=float)
    for item in tqdm.tqdm(valid_loader):
        inputs, target = item
        output = model(inputs)
        rmses += np.sum((
            np.square(target.detach().cpu().numpy() - output.detach().cpu().numpy())
        ) * (target.detach().cpu().numpy() != -1), axis=(0, 2, 3, 4))
    rmses /= len(valid_loader)
    return np.mean(np.sqrt(rmses))


def process_test(model, test_loader, output_file='../output.hdf5'):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    for index, item in tqdm.tqdm(enumerate(test_loader)):
        (inputs, last_input_timestamp), _ = item
        output = model(inputs)
        with h5py.File(output_file, mode='a') as f_out:
            for ind in range(output.shape[1]):
                timestamp_out = str(int(last_input_timestamp[-1]) + 600 * (ind + 1))
                f_out.create_group(timestamp_out)
                f_out[timestamp_out].create_dataset(
                    'intensity',
                    data=output[0, ind, 0].detach().cpu().numpy()
                )

In [ ]:
train_loaders = prepare_loaders()
valid_loader = prepare_valid_loader()

In [ ]:
model = ConvLSTMModel()

In [ ]:
trainer = L.Trainer(
    max_epochs=1
)
trainer.fit(model, train_loaders[0])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type    | Params
----------------------------------
0 | model | Seq2Seq | 201 K 
----------------------------------
201 K     Trainable params
0         Non-trainable params
201 K     Total params
0.807     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name  | Type    | Params
---------------------------

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:

trainer = L.Trainer(
    max_epochs=1
)
trainer.fit(model, train_loaders[1])


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type    | Params
----------------------------------
0 | model | Seq2Seq | 201 K 
----------------------------------
201 K     Trainable params
0         Non-trainable params
201 K     Total params
0.807     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name  | Type    | Params
---------------------------

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
evaluate_on_val(model, valid_loader) #168.3


100%|██████████| 279/279 [04:45<00:00,  1.02s/it]


41431.4073597233

In [ ]:
trainer = L.Trainer(
    max_epochs=1
)
trainer.fit(model, train_loaders[2])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type    | Params
----------------------------------
0 | model | Seq2Seq | 201 K 
----------------------------------
201 K     Trainable params
0         Non-trainable params
201 K     Total params
0.807     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name  | Type    | Params
---------------------------

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
device = torch.device('cuda:0')
model.to(device)

ConvLSTMModel(
  (model): Seq2Seq(
    (sequential): Sequential(
      (convlstm1): ConvLSTM(
        (convLSTMCell): ConvLSTMCell(
          (conv_1): Conv2d(33, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv_2): Conv2d(33, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv_3): Conv2d(33, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv_wind): Conv2d(42, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv_int): Conv2d(33, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
    )
    (conv): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)

In [ ]:
valid_loader = prepare_valid_loader()

In [ ]:
evaluate_on_val(model, valid_loader) #165.8

In [ ]:
import gc
del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer = L.Trainer(
    max_epochs=1
)
trainer.fit(model, train_loaders[3])

In [ ]:
model.to(device)
evaluate_on_val(model, valid_loader)

In [ ]:
trainer.fit(model, train_loaders[4])

In [ ]:
model.to(device)
evaluate_on_val(model, valid_loader)

In [ ]:
torch.save(model, '/content/drive/MyDrive/ML Cup 2023 Weather/2_epochs_full_data.pt')

In [ ]:
valid_loader = prepare_valid_loader()

In [ ]:
print(torch.cuda.is_available())

In [ ]:
evaluate_on_val(model, valid_loader)

In [ ]:
def prepare_test_loader(test_batch_size=1):
    test_dataset = RadarDataset(['/content/drive/MyDrive/ML Cup 2023 Weather/2022-test-public.hdf5'], out_seq_len=0, with_time=True)
    test_loader = data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=2)
    return test_loader

In [ ]:
#torch.save(model, '/content/drive/MyDrive/ML Cup 2023 Weather/model_conv_2_with_2_epochs.pt')

In [ ]:
test_loader = prepare_test_loader()
process_test(model, test_loader, output_file='/content/drive/MyDrive/ML Cup 2023 Weather/kernel_64_3_data.hdf5')

96it [00:17,  5.44it/s]


In [ ]:
import torch
import torch.nn as nn

# Define the size of the activation map
activation_map_size = (10, 252, 252)

# Define the number of output channels for the convolutional layer
num_output_channels = 32

# Define the kernel size for the convolutional layer
kernel_size = 3

# Create a random activation map
activation_map = torch.randn(activation_map_size)

# Define the convolutional layer
conv_layer = nn.Conv2d(activation_map_size[0], num_output_channels, kernel_size, padding=1)

# Apply the convolutional layer to the activation map
output = conv_layer(activation_map)

# Print the size of the output activation map
print(output.size())

torch.Size([32, 252, 252])
